# Subir archivo


In [ ]:
from google.colab import files
uploaded = files.upload()
#rename to data.xlsx
import os

uploaded_filename = list(uploaded.keys())[0]
os.rename(uploaded_filename, "data.xlsx")

Saving data.xlsx to data.xlsx


# 1. Instalar dependencias

In [ ]:
!pip install streamlit plotly pandas streamlit_elements

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 899.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.9 MB/s eta 0:00:00


# 2. Instalar local tunnels

In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 3s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

# 3. Crear funciones necesarias

In [ ]:
%%writefile utils.py
from typing import Tuple, List, Dict, Any, Optional
import pandas as pd
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objects import Figure

def prepareDataTable(plot_data, is_higher_better):
    """Prepare data for display in a table with semaphore status.

    Parameters:
    -----------
    plot_data : pd.DataFrame
        DataFrame with KPI, Meta, and Aceptacion columns
    is_higher_better : bool
        Whether higher values are better for this KPI

    Returns:
    --------
    pd.DataFrame
        Data formatted for display in a table
    """
    # Create a copy to avoid modifying the original
    display_data = plot_data.copy()

    # Format date column if present
    if 'Fecha' in display_data.columns:
        display_data['Fecha'] = display_data['Fecha'].dt.strftime('%Y/%m')

    # Apply function to create semaphore status
    display_data['Estado'] = display_data.apply(
        lambda row: getSemaphore(row, is_higher_better),
        axis=1
    )

    return display_data

def loadData(filePath: str = "data.xlsx") -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Load data from Excel file and return the necessary dataframes."""
    try:
        mediciones = pd.read_excel(filePath, sheet_name=0)
        errores = pd.read_excel(filePath, sheet_name=1)
        tiempoDeRespuesta = pd.read_excel(filePath, sheet_name=2)
        disponibilidad = pd.read_excel(filePath, sheet_name=3)
        tiquetesConErrores = pd.read_excel(filePath, sheet_name=4)
    except FileNotFoundError:
        raise FileNotFoundError("El archivo data.xlsx no se encuentra en la ruta especificada.")
    except ValueError:
        raise ValueError("El archivo data.xlsx no contiene las hojas necesarias.")
    except Exception as e:
        raise Exception(f"Error al cargar el archivo: {e}")

    try:
        mediciones['Fecha_Inicio_Mes'] = pd.to_datetime(mediciones['Fecha_Inicio_Mes'])
        mediciones['Fecha_Fin_Mes'] = pd.to_datetime(mediciones['Fecha_Fin_Mes'])
        tiempoDeRespuesta['Fecha_Inicio_Mes'] = pd.to_datetime(tiempoDeRespuesta['Fecha_Inicio_Mes'])
        tiempoDeRespuesta['Fecha_Fin_Mes'] = pd.to_datetime(tiempoDeRespuesta['Fecha_Fin_Mes'])
        disponibilidad['Fecha_Inicio_Mes'] = pd.to_datetime(disponibilidad['Fecha_Inicio_Mes'])
        disponibilidad['Fecha_Fin_Mes'] = pd.to_datetime(disponibilidad['Fecha_Fin_Mes'])
    except KeyError as e:
        raise KeyError(f"Error al convertir columnas a datetime: {e}")
    except Exception as e:
        raise Exception(f"Error inesperado: {e}")

    return mediciones, tiempoDeRespuesta, disponibilidad, tiquetesConErrores, errores


def preprocessErrores(errores: pd.DataFrame) -> pd.DataFrame:
    """Process the errores DataFrame to add date column if needed."""
    if 'Fecha' not in errores.columns and 'Año' in errores.columns and 'Trimestre' in errores.columns:
        # Mapear trimestres a meses (usando el primer mes de cada trimestre)
        trimestre_a_mes = {
            'Trim.1': 1,
            'Trim.2': 4,
            'Trim.3': 7,
            'Trim.4': 10
        }

        # Crear una columna de fecha
        errores['Fecha'] = errores.apply(
            lambda row: pd.Timestamp(year=row['Año'], month=trimestre_a_mes[row['Trimestre']], day=1),
            axis=1
        )

        # Ordenar por fecha
        errores = errores.sort_values(['Aplicación', 'Criticidad', 'Fecha'])

    return errores


def getSemaphore(row, higher_is_better):
    """Return semaphore icon based on KPI, meta and acceptance values."""
    kpi_value = row['KPI']
    meta_value = row['Meta']
    acceptance_value = row['Aceptacion']

    if higher_is_better:
        # Higher is better
        if kpi_value >= meta_value:
            return "🟢 Cumplido"
        elif kpi_value >= acceptance_value:
            return "🟡 Aceptado"
        else:
            return "🔴 Incumplido"
    else:
        # Lower is better
        if kpi_value <= meta_value:
            return "🟢 Cumplido"
        elif kpi_value <= acceptance_value:
            return "🟡 Aceptado"
        else:
            return "🔴 Incumplido"

def setupFilters(mediciones):
    """Set up filters in sidebar and return selected values and filtered data."""
    services = mediciones['Aplicación'].unique().tolist()

    years = sorted(mediciones['Fecha_Inicio_Mes'].dt.year.unique().tolist())
    months = list(range(1, 13))

    st.sidebar.header("Dashboard de KPIs")
    st.sidebar.subheader("Filtros")
    selected_service = st.sidebar.selectbox("Seleccionar servicio", services)
    selected_kpi = st.sidebar.selectbox("Seleccionar KPI", {"Tiempo Respuesta": "Tiempo Respuesta", "Disponibilidad": "Horas_Disp_reales_mes", "Tiquetes con Errores": "Tiquetes con Errores"})

    selected_year = st.sidebar.selectbox("Seleccionar año", ["Todos"] + years)
    selected_month = st.sidebar.selectbox("Seleccionar mes", ["Todos"] + [f"{m:02d}" for m in months])

    service_data = mediciones[mediciones['Aplicación'] == selected_service]

    if selected_year != "Todos":
        service_data = service_data[service_data['Fecha_Inicio_Mes'].dt.year == selected_year]

    if selected_month != "Todos":
        selected_month_int = int(selected_month)
        service_data = service_data[service_data['Fecha_Inicio_Mes'].dt.month == selected_month_int]

    service_data = service_data.sort_values('Fecha_Inicio_Mes')

    return selected_service, selected_kpi, service_data

def calculateKpiValues(service_data, kpi_name, tiempoDeRespuesta, disponibilidad, tiquetesConErrores):
    """Calculate KPI values for the specified KPI type."""
    if kpi_name == "Tiempo Respuesta":
        kpi_values = service_data["Tiempo Respuesta"].tolist()
        meta_values = [tiempoDeRespuesta.loc[tiempoDeRespuesta['Fecha_Inicio_Mes'] == date, 'Meta_ToR'].values[0]
                      if date in tiempoDeRespuesta['Fecha_Inicio_Mes'].values else None
                      for date in service_data['Fecha_Inicio_Mes']]
        nivel_aceptable = [tiempoDeRespuesta.loc[tiempoDeRespuesta['Fecha_Inicio_Mes'] == date, 'Nivel_Aceptable'].values[0]
                           if date in tiempoDeRespuesta['Fecha_Inicio_Mes'].values else None
                           for date in service_data['Fecha_Inicio_Mes']]
        y_axis_label = "Tiempo de Respuesta"

    elif kpi_name == "Disponibilidad":
        # Convert availability to percentage (multiply by 100)
        kpi_values = ((service_data["Horas_Disp_reales_mes"] /
                      (service_data["Horas_Disp_Mes"] - service_data["Horas_Indisp_Progr_mes"])) * 100).tolist()
        meta_values = [disponibilidad.loc[disponibilidad['Fecha_Inicio_Mes'] == date, 'Meta_disp'].values[0] * 100
                      if date in disponibilidad['Fecha_Inicio_Mes'].values else None
                      for date in service_data['Fecha_Inicio_Mes']]
        nivel_aceptable = [disponibilidad.loc[disponibilidad['Fecha_Inicio_Mes'] == date, 'Nivel_Aceptable'].values[0] * 100
                           if date in disponibilidad['Fecha_Inicio_Mes'].values else None
                           for date in service_data['Fecha_Inicio_Mes']]
        y_axis_label = "Disponibilidad (%)"

    elif kpi_name == "Tiquetes con Errores":
        kpi_values = service_data["Tiquetes con Errores"].tolist()
        current_year = service_data['Fecha_Inicio_Mes'].dt.year.iloc[0] if not service_data.empty else 2020
        criticality = "Alta"
        meta_values = [tiquetesConErrores.loc[(tiquetesConErrores['Criticidad'] == criticality) &
                                              (tiquetesConErrores['Año'] == current_year), 'Meta'].values[0]
                       if not tiquetesConErrores[(tiquetesConErrores['Criticidad'] == criticality) &
                                                (tiquetesConErrores['Año'] == current_year)].empty else None]
        nivel_aceptable = [meta_values[0]] if meta_values else [None]
        y_axis_label = "Numero de Tiquetes con Errores"

    return kpi_values, meta_values, nivel_aceptable, y_axis_label

def calculateTCEWithErroresTable(service_data, selected_kpi, errores, selected_service, tiquetesConErrores=None, criticality=None):
    """Calculate ticket error KPI values using the errores table."""
    if selected_kpi == "Tiquetes con Errores":
        # If tiquetesConErrores is not provided, default meta values will be used
        if tiquetesConErrores is None:
            st.warning("Datos de metas para tiquetes no proporcionados. Se usarán valores predeterminados.")

        # Filtrar los datos de errores para el servicio seleccionado
        service_errors = errores[errores['Aplicación'] == selected_service].sort_values('Fecha')

        if service_errors.empty:
            return [], [], [], "Porcentaje de Tiquetes Resueltos"

        # Get date range from service_data to filter errors by date
        if not service_data.empty:
            # Get min and max dates from service_data
            min_date = service_data['Fecha_Inicio_Mes'].min()
            max_date = service_data['Fecha_Inicio_Mes'].max()

            # Filter errors by date range
            if 'Fecha' in service_errors.columns:
                service_errors = service_errors[
                    (service_errors['Fecha'] >= min_date) &
                    (service_errors['Fecha'] <= max_date)
                ]
            elif 'Año' in service_errors.columns:
                # If we only have year, filter by year
                min_year = min_date.year
                max_year = max_date.year
                service_errors = service_errors[
                    (service_errors['Año'] >= min_year) &
                    (service_errors['Año'] <= max_year)
                ]

        # Si después del filtrado no hay datos, devolver listas vacías
        if service_errors.empty:
            return [], [], [], "Porcentaje de Tiquetes Resueltos"

        # Obtener la criticidad disponible (si no se especifica, usar la primera que aparece)
        criticalities = service_errors['Criticidad'].unique()

        # Si no hay criticidades disponibles, devolver listas vacías
        if len(criticalities) == 0:
            return [], [], [], "Porcentaje de Tiquetes Resueltos"

        # Si se proporciona una criticidad específica, usarla
        if criticality is not None:
            if criticality in criticalities:
                selected_criticality = criticality
            else:
                return [], [], [], "Porcentaje de Tiquetes Resueltos"
        # Permitir al usuario seleccionar la criticidad si hay más de una y no se proporciona una específica
        elif len(criticalities) > 1 and criticality is None:
            selected_criticality = st.sidebar.selectbox("Seleccionar Criticidad", criticalities, key=f"crit_select_{selected_service}")
        else:
            selected_criticality = criticalities[0]

        # Filtrar por criticidad
        service_errors_by_crit = service_errors[service_errors['Criticidad'] == selected_criticality]

        if service_errors_by_crit.empty:
            return [], [], [], "Porcentaje de Tiquetes Resueltos"

        # Calcular el KPI como porcentaje de tiquetes resueltos
        kpi_values = []
        for i in range(len(service_errors_by_crit)):
            total_errors = service_errors_by_crit['Cant_Tiq_Error_Trim'].iloc[i]
            closed_errors = service_errors_by_crit['Cant_Tiq_Error_Cerr'].iloc[i]

            # Calcular porcentaje de tiquetes resueltos
            if total_errors == 0:
                pct_resolved = 100  # Si no hay errores, 100% resueltos
            else:
                pct_resolved = (closed_errors / total_errors) * 100

            kpi_values.append(pct_resolved)

        # Para el servicio seleccionado, obtener la meta de tiquetes según criticidad y año
        years = service_errors_by_crit['Año'].unique()
        meta_values = []
        nivel_aceptable = []

        for year in years:
            if tiquetesConErrores is not None:
                # Buscar en el dataframe tiquetesConErrores para obtener los valores de meta
                meta_row = tiquetesConErrores[
                    (tiquetesConErrores['Criticidad'] == selected_criticality) &
                    (tiquetesConErrores['Año'] == year)
                ]

                if not meta_row.empty:
                    # FIXED: Convert Meta to percentage (multiply by 100) if it's a ratio
                    meta = meta_row['Meta'].values[0]
                    if meta <= 1:  # If meta is expressed as a ratio (0-1)
                        meta = meta * 100  # Convert to percentage (0-100)
                    meta_values.extend([meta] * 4)  # Un valor para cada trimestre

                    # Check if 'Nivel_Aceptable' column exists in tiquetesConErrores
                    if 'Nivel_Aceptable' in meta_row.columns:
                        accept = meta_row['Nivel_Aceptable'].values[0]
                        if accept <= 1:  # If acceptance is expressed as a ratio (0-1)
                            accept = accept * 100  # Convert to percentage (0-100)
                        nivel_aceptable.extend([accept] * 4)
                    else:
                        # Use default of 80% of meta
                        nivel_aceptable.extend([meta * 0.8] * 4)
                else:
                    # Valores predeterminados si no se encuentra - already in percentage (0-100)
                    meta_values.extend([80] * 4)  # 80% resueltos como meta
                    nivel_aceptable.extend([64] * 4)  # 64% como aceptable
            else:
                # Si no se proporciona tiquetesConErrores, usar valores predeterminados - already in percentage (0-100)
                meta_values.extend([80] * 4)  # 80% resueltos como meta
                nivel_aceptable.extend([64] * 4)  # 64% como aceptable

        # Ajustar las listas para que coincidan en longitud con kpi_values
        meta_values = meta_values[:len(kpi_values)]
        nivel_aceptable = nivel_aceptable[:len(kpi_values)]

        return kpi_values, meta_values, nivel_aceptable, f"Porcentaje de Tiquetes Resueltos ({selected_criticality})"

    # Para otros KPIs, devolver datos vacíos
    return [], [], [], "Unknown KPI"

def checkTypeOfKpi(plot_data):
    """Determine if a KPI is better when higher or lower."""
    higher_is_better = None
    if len(plot_data) > 0:
        first_meta = plot_data['Meta'].iloc[0]
        first_acceptable = plot_data['Aceptacion'].iloc[0]

        if first_meta is not None and first_acceptable is not None:
            higher_is_better = first_meta > first_acceptable
    return higher_is_better

def plotKpiLineChart(plot_data, selected_kpi, selected_service, direction_label, y_axis_label) -> Figure:
    """Create a line chart for KPI data."""
    # Create the line plot using Plotly
    fig = px.line(plot_data, x='Fecha', y=['KPI', 'Meta', 'Aceptacion'],
                title=f"{selected_kpi} para {selected_service}{direction_label}",
                labels={'value': y_axis_label, 'Fecha': 'Mes', 'variable': 'Metrica'})

    # Update line styles for better visualization
    fig.update_traces(mode='lines+markers')
    fig.update_layout(
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        height=400,
        xaxis=dict(
            tickformat="%Y/%m"
        )
    )
    return fig

def createGauge(value, meta, acceptance, title, is_higher_better):
    """Create a gauge chart for KPI visualization."""
    if is_higher_better:
        # For metrics where higher is better (e.g. availability)
        steps = [
            {'range': [0, acceptance], 'color': 'red'},
            {'range': [acceptance, meta], 'color': 'yellow'},
            {'range': [meta, max(110, 1.1 * meta)], 'color': 'green'}
        ]
        max_val = max(110, 1.1 * meta)  # Ensure max is at least 110 for percentages
    else:
        # For metrics where lower is better (e.g. response time, error tickets)
        steps = [
            {'range': [0, meta], 'color': 'green'},
            {'range': [meta, acceptance], 'color': 'yellow'},
            {'range': [acceptance, 2 * acceptance], 'color': 'red'}
        ]
        max_val = 2 * acceptance

    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=value,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': title},
        gauge={
            'axis': {'range': [0, max_val]},
            'bar': {'color': "#1f77b4"},
            'steps': steps,
            'threshold': {
                'line': {'color': "black", 'width': 3},
                'thickness': 0.75,
                'value': meta
            }
        }
    ))

    # Add % sign for percentage metrics
    if title == "Disponibilidad" or title == "% Tiquetes Resueltos":
        fig.update_traces(number={'suffix': '%'})

    fig.update_layout(height=200, margin=dict(l=30, r=30, t=50, b=30))
    return fig


def getAllServicesKpi(kpi_name, mediciones, tiempoDeRespuesta, disponibilidad, tiquetesConErrores):
    """Get KPI values for all services for comparison."""
    services = mediciones['Aplicación'].unique().tolist()
    latest_values = []
    meta_values = []
    acceptance_values = []

    # Get the latest date for each service
    for service in services:
        service_data = mediciones[mediciones['Aplicación'] == service].sort_values('Fecha_Inicio_Mes')

        if service_data.empty:
            continue

        # Calculate KPI values for this service
        kpi_vals, meta_vals, accept_vals, _ = calculateKpiValues(
            service_data, kpi_name, tiempoDeRespuesta, disponibilidad, tiquetesConErrores
        )

        if kpi_vals and len(kpi_vals) > 0:
            latest_values.append((service, kpi_vals[-1]))
            meta_values.append(meta_vals[-1] if len(meta_vals) > 0 else None)
            acceptance_values.append(accept_vals[-1] if len(accept_vals) > 0 else None)

    return pd.DataFrame({
        'Servicio': [item[0] for item in latest_values],
        'KPI': [item[1] for item in latest_values],
        'Meta': meta_values,
        'Aceptacion': acceptance_values
    })

def getAllServicesTicketsData(service_data, errores, tiquetesConErrores):
    """Get ticket data for all services."""
    all_services_data = pd.DataFrame()
    services = errores['Aplicación'].unique()

    for service in services:
        # Para cada servicio, obtener todas las criticidades
        service_criticalities = errores[errores['Aplicación'] == service]['Criticidad'].unique()

        for crit in service_criticalities:
            # Calcular el KPI para este servicio y criticidad
            service_kpi, service_meta, service_accept, _ = calculateTCEWithErroresTable(
                service_data, "Tiquetes con Errores", errores, service, tiquetesConErrores, crit
            )

            # Si hay datos, agregar a all_services_data
            if len(service_kpi) > 0:
                # Usar el último valor como el actual
                new_row = pd.DataFrame({
                    'Servicio': [f"{service} ({crit})"],
                    'KPI': [service_kpi[-1]],
                    'Meta': [service_meta[-1]],
                    'Aceptacion': [service_accept[-1]]
                })
                all_services_data = pd.concat([all_services_data, new_row], ignore_index=True)

    return all_services_data

def getBarColor(row, is_higher_better):
    """Determine color for bar charts based on KPI performance."""
    kpi_val = row['KPI']
    meta_val = row['Meta']
    accept_val = row['Aceptacion']

    # If meta or acceptance values are missing, return a neutral color
    if meta_val is None or accept_val is None:
        return 'gray'

    if is_higher_better:
        # Higher is better (like availability or ticket resolution %)
        if kpi_val >= meta_val:
            return 'green'
        elif kpi_val >= accept_val:
            return 'gold'
        else:
            return 'red'
    else:
        # Lower is better (like response time)
        if kpi_val <= meta_val:
            return 'green'
        elif kpi_val <= accept_val:
            return 'gold'
        else:
            return 'red'

def createBarChart(all_services_data, title, selected_service, direction_label, y_axis_label):
    """Create bar chart for service comparison."""
    bar_fig = px.bar(
        all_services_data,
        x='Servicio',
        y='KPI',
        height=400,
        title=f"Comparación de {title} por servicio{direction_label}",
        labels={'KPI': y_axis_label, 'Servicio': 'Servicio'}
    )

    # Customize bar colors
    bar_fig.update_traces(marker_color=all_services_data['Color'])

    # Add reference lines for meta and acceptance thresholds
    if not all_services_data.empty:
        if all_services_data['Meta'].iloc[0] is not None:
            bar_fig.add_shape(
                type='line',
                x0=-0.5,
                x1=len(all_services_data) - 0.5,
                y0=all_services_data['Meta'].iloc[0],
                y1=all_services_data['Meta'].iloc[0],
                line=dict(color='blue', dash='dash', width=2),
                name='Meta'
            )

        if all_services_data['Aceptacion'].iloc[0] is not None:
            bar_fig.add_shape(
                type='line',
                x0=-0.5,
                x1=len(all_services_data) - 0.5,
                y0=all_services_data['Aceptacion'].iloc[0],
                y1=all_services_data['Aceptacion'].iloc[0],
                line=dict(color='orange', dash='dot', width=2),
                name='Nivel Aceptable'
            )

    # Highlight the selected service
    if selected_service in all_services_data['Servicio'].values:
        idx = all_services_data.index[all_services_data['Servicio'] == selected_service].tolist()[0]
        bar_fig.add_annotation(
            x=selected_service,
            y=all_services_data.loc[idx, 'KPI'],
            text="Seleccionado",
            showarrow=True,
            arrowhead=1,
            ax=0,
            ay=-40
        )
    return bar_fig

def create_plot_data(selected_kpi, service_data, kpi_values, meta_values, nivel_aceptable, errores, selected_service, criticality=None):
    """Create DataFrame for plotting KPI data."""
    if len(kpi_values) > 0:
        if selected_kpi == "Tiquetes con Errores" and 'Fecha' in errores.columns:
            service_errors = errores[errores['Aplicación'] == selected_service].sort_values('Fecha')

            # Filtrar por criticidad si se proporciona
            if criticality is not None:
                service_errors = service_errors[service_errors['Criticidad'] == criticality]

            # Usar las fechas de los errores
            plot_data = pd.DataFrame({
                'Fecha': service_errors['Fecha'].values[:len(kpi_values)],
                'KPI': kpi_values,
                'Meta': meta_values,
                'Aceptacion': nivel_aceptable
            })
        else:
            # Usar las fechas del service_data
            plot_data = pd.DataFrame({
                'Fecha': service_data['Fecha_Inicio_Mes'][:len(kpi_values)],
                'KPI': kpi_values,
                'Meta': meta_values * len(kpi_values) if len(meta_values) == 1 else meta_values,
                'Aceptacion': nivel_aceptable * len(kpi_values) if len(nivel_aceptable) == 1 else nivel_aceptable
            })
    else:
        # Crear un DataFrame vacío si no hay datos
        plot_data = pd.DataFrame({
            'Fecha': [],
            'KPI': [],
            'Meta': [],
            'Aceptacion': []
        })

    # Eliminar filas con fechas duplicadas (se toma la primera)
    plot_data = plot_data.drop_duplicates(subset=['Fecha'])

    # Ordenar los datos por fecha
    plot_data = plot_data.sort_values('Fecha')

    return plot_data

def calculateAllKpiValues(service_data, errores, tiempoDeRespuesta, disponibilidad, tiquetesConErrores, selected_service):
    """Calculate values for all KPIs at once."""
    # Para Tiempo de Respuesta
    tor_service_data = service_data.copy()
    ToR = calculateKpiValues(tor_service_data, "Tiempo Respuesta", tiempoDeRespuesta, disponibilidad, tiquetesConErrores)
    kpi_ToR = ToR[0]
    meta_ToR = ToR[1]
    acceptance_ToR = ToR[2]

    # Para Disponibilidad
    disp_service_data = service_data.copy()
    Disp = calculateKpiValues(disp_service_data, "Disponibilidad", tiempoDeRespuesta, disponibilidad, tiquetesConErrores)
    kpi_Disp = Disp[0]
    meta_Disp = Disp[1]
    acceptance_Disp = Disp[2]

    # Para Tiquetes con Errores - pasando tiquetesConErrores como parámetro
    Tiq = calculateTCEWithErroresTable(service_data, "Tiquetes con Errores", errores, selected_service, tiquetesConErrores)
    kpi_Tiquetes = Tiq[0]
    meta_Tiquetes = Tiq[1]
    acceptance_Tiquetes = Tiq[2]

    return {
        'ToR': {
            'kpi': kpi_ToR,
            'meta': meta_ToR,
            'acceptance': acceptance_ToR
        },
        'Disp': {
            'kpi': kpi_Disp,
            'meta': meta_Disp,
            'acceptance': acceptance_Disp
        },
        'Tiq': {
            'kpi': kpi_Tiquetes,
            'meta': meta_Tiquetes,
            'acceptance': acceptance_Tiquetes
        }
    }

Writing utils.py


# 4. Crear UI

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from utils import (
    loadData, preprocessErrores, setupFilters, calculateAllKpiValues, create_plot_data,
    checkTypeOfKpi, plotKpiLineChart, createGauge, getAllServicesKpi, getAllServicesTicketsData,
    createBarChart, getBarColor, prepareDataTable, getSemaphore
)

# Configurar la página de Streamlit
st.set_page_config(
    page_title="Dashboard | KPIs",
    page_icon="📊",
    layout="wide"
)

# Cargar y preprocesar los datos
mediciones, tiempoDeRespuesta, disponibilidad, tiquetesConErrores, errores = loadData()
errores = preprocessErrores(errores)

# Configurar filtros y obtener datos filtrados
selected_service, selected_kpi, service_data = setupFilters(mediciones)

# Calcular valores para todos los KPIs independientemente del KPI seleccionado
all_kpi_values = calculateAllKpiValues(
    service_data, errores, tiempoDeRespuesta, disponibilidad, tiquetesConErrores, selected_service
)

# Obtener los valores de KPI para cada tipo
kpi_ToR = all_kpi_values['ToR']['kpi']
meta_ToR = all_kpi_values['ToR']['meta']
acceptance_ToR = all_kpi_values['ToR']['acceptance']

kpi_Disp = all_kpi_values['Disp']['kpi']
meta_Disp = all_kpi_values['Disp']['meta']
acceptance_Disp = all_kpi_values['Disp']['acceptance']

kpi_Tiquetes = all_kpi_values['Tiq']['kpi']
meta_Tiquetes = all_kpi_values['Tiq']['meta']
acceptance_Tiquetes = all_kpi_values['Tiq']['acceptance']

# Crear el DataFrame para la gráfica para el KPI seleccionado
plot_data = create_plot_data(
    selected_kpi, service_data, kpi_ToR if selected_kpi == "Tiempo Respuesta" else kpi_Disp if selected_kpi == "Disponibilidad" else kpi_Tiquetes,
    meta_ToR if selected_kpi == "Tiempo Respuesta" else meta_Disp if selected_kpi == "Disponibilidad" else meta_Tiquetes,
    acceptance_ToR if selected_kpi == "Tiempo Respuesta" else acceptance_Disp if selected_kpi == "Disponibilidad" else acceptance_Tiquetes,
    errores, selected_service
)

# Determinar si el KPI es mejor cuando es más alto o más bajo
is_higher_is_better = selected_kpi != "Tiempo Respuesta"  # Forced logic: ToR is lower-better, others are higher-better

# Crear la etiqueta de tipo de KPI
direction_label = " (Más es mejor)" if is_higher_is_better else " (Menos es mejor)"

# Crear layout de columnas para el dashboard
main_col, metrics_col = st.columns([7, 2])

# Columna principal para gráficos y métricas principales
with main_col:
    # Mostrar estado actual de los KPI contra el valor anterior
    col1, col2, col3 = st.columns(3)

    # Metricas de Tiempo de Respuesta
    if len(kpi_ToR) >= 2:
        tor_diff = kpi_ToR[-1] - kpi_ToR[-2]
        col1.metric(
            f"Tiempo de Respuesta (s)",
            f"{kpi_ToR[-1]:.2f}s",
            f"{tor_diff:.2f}s",
            delta_color="inverse"
        )
    else:
        col1.metric(f"Tiempo de Respuesta (s)", "N/A", "0s", delta_color="inverse")

    # Metricas de Disponibilidad
    if len(kpi_Disp) >= 2:
        disp_diff = kpi_Disp[-1] - kpi_Disp[-2]
        col2.metric(
            f"Disponibilidad (%)",
            f"{kpi_Disp[-1]:.2f}%",
            f"{disp_diff:.2f}%",
            delta_color="normal"
        )
    else:
        col2.metric(f"Disponibilidad (%)", "N/A", "0%", delta_color="normal")

    # Metricas de Tiquetes Resueltos
    if len(kpi_Tiquetes) >= 2:
        tiq_diff = kpi_Tiquetes[-1] - kpi_Tiquetes[-2]
        col3.metric(
            f"Tiquetes Resueltos (%)",
            f"{kpi_Tiquetes[-1]:.2f}%",
            f"{tiq_diff:.2f}%",
            delta_color="normal"
        )
    else:
        col3.metric(f"Tiquetes Resueltos (%)", "N/A", "0%", delta_color="normal")

    # Gráfica de línea para el KPI seleccionado con datos historicos
    if not plot_data.empty:
        y_axis_label = "Tiempo de Respuesta (s)" if selected_kpi == "Tiempo Respuesta" else "Disponibilidad (%)" if selected_kpi == "Disponibilidad" else "Porcentaje de Tiquetes Resueltos"
        line_chart = plotKpiLineChart(plot_data, selected_kpi, selected_service, direction_label, y_axis_label)
        st.plotly_chart(line_chart, use_container_width=True)
    else:
        st.info("No hay datos suficientes para mostrar la gráfica.")

# Columna de métricas adicionales
with metrics_col:
    st.subheader("Medidores de KPI")

    # Velocimetro para Tiempo de Respuesta
    if len(kpi_ToR) > 0:
        tor_gauge = createGauge(kpi_ToR[-1], meta_ToR[-1], acceptance_ToR[-1], "Tiempo de Respuesta", False)
        st.plotly_chart(tor_gauge, use_container_width=True)
    else:
        st.info("No hay datos para el medidor de Tiempo de Respuesta.")

    # Velocimetro para Disponibilidad
    if len(kpi_Disp) > 0:
        disp_gauge = createGauge(kpi_Disp[-1], meta_Disp[-1], acceptance_Disp[-1], "Disponibilidad", True)
        st.plotly_chart(disp_gauge, use_container_width=True)
    else:
        st.info("No hay datos para el medidor de Disponibilidad.")

    # Velocimetro para Tiquetes Resueltos
    if len(kpi_Tiquetes) > 0:
        tiq_gauge = createGauge(kpi_Tiquetes[-1], meta_Tiquetes[-1], acceptance_Tiquetes[-1], "Tiquetes Resueltos", True)
        st.plotly_chart(tiq_gauge, use_container_width=True)
    else:
        st.info("No hay datos para el medidor de Tiquetes Resueltos.")

# Comparación entre servicios
st.subheader("Comparación entre Servicios")
if selected_kpi == "Tiquetes con Errores":
    all_services_data = getAllServicesTicketsData(service_data, errores, tiquetesConErrores)
else:
    all_services_data = getAllServicesKpi(selected_kpi, mediciones, tiempoDeRespuesta, disponibilidad, tiquetesConErrores)

if not all_services_data.empty:
    is_higher_better = selected_kpi != "Tiempo Respuesta"

    # Aplicar el color a las barras con base a el cumplimiento de la meta
    all_services_data['Color'] = all_services_data.apply(
        lambda row: getBarColor(row, is_higher_better),
        axis=1
    )

    # Crear y mostrar la gráfica de barras
    bar_chart = createBarChart(all_services_data, selected_kpi, selected_service, direction_label, y_axis_label)
    st.plotly_chart(bar_chart, use_container_width=True)

    # Sección de datos por servicio
    st.subheader("Datos por Servicio")

    # Crear un datafrmame para mostrar los datos de todos los servicios
    display_data = all_services_data.copy()
    if 'Color' in display_data.columns:
        display_data = display_data.drop(columns=['Color'])

    # Aplicar la funcion para determinar el estado de cada servicio
    display_data['Estado'] = display_data.apply(
        lambda row: getSemaphore(row, is_higher_better),
        axis=1
    )

    # Dar formato de porcentaje a los valores de KPI, Meta y Aceptacion
    if selected_kpi != "Tiempo Respuesta":
        display_data['KPI'] = display_data['KPI'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A")
        display_data['Meta'] = display_data['Meta'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A")
        display_data['Aceptacion'] = display_data['Aceptacion'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A")
    else:
        display_data['KPI'] = display_data['KPI'].apply(lambda x: f"{x:.2f}s" if pd.notnull(x) else "N/A")
        display_data['Meta'] = display_data['Meta'].apply(lambda x: f"{x:.2f}s" if pd.notnull(x) else "N/A")
        display_data['Aceptacion'] = display_data['Aceptacion'].apply(lambda x: f"{x:.2f}s" if pd.notnull(x) else "N/A")

    # Mostrar la tabla con los semáforos
    with st.expander("Comparación entre servicios", expanded=True):
        st.dataframe(display_data, use_container_width=True)
else:
    st.info("No hay datos suficientes para la comparación entre servicios.")

# Tabla detallada del servicio seleccionado
if not plot_data.empty:
    # Preparar los datos para la tabla
    service_table_data = prepareDataTable(plot_data, is_higher_better)

    # Dar formato con base en el tipo de KPI
    if selected_kpi != "Tiempo Respuesta":
        service_table_data['KPI'] = service_table_data['KPI'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A")
        service_table_data['Meta'] = service_table_data['Meta'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A")
        service_table_data['Aceptacion'] = service_table_data['Aceptacion'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A")
    else:
        service_table_data['KPI'] = service_table_data['KPI'].apply(lambda x: f"{x:.2f}s" if pd.notnull(x) else "N/A")
        service_table_data['Meta'] = service_table_data['Meta'].apply(lambda x: f"{x:.2f}s" if pd.notnull(x) else "N/A")
        service_table_data['Aceptacion'] = service_table_data['Aceptacion'].apply(lambda x: f"{x:.2f}s" if pd.notnull(x) else "N/A")

    # Mostrar la tabla con los datos del servicio seleccionado
    with st.expander("Datos historicos del servicio seleccionado", expanded=True):
        st.dataframe(service_table_data, use_container_width=True)
else:
    st.info("No hay datos suficientes para mostrar la tabla del servicio seleccionado.")

Writing app.py


# 5. Obtener IP
> Utilizar como contraseña de la url en el proximo paso

In [ ]:
!wget -q -O - ipv4.icanhazip.com

104.155.213.97


# 6. Exponer tunel

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙

your url is: https://tough-lizards-repair.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.155.213.97:8501

  Stopping...
^C
